In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!mv kaggle.json ~/.kaggle/

In [ ]:
!!kaggle competitions download -c deepfake-detection-challenge

!unzip deepfake-detection-challenge.zip

In [ ]:
import shutil
import os
import json

def filter_and_copy_videos(metadata, source_dir, target_dir, num_real=5, num_fake=5):
    # Create target directory if it does not exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Extract real and fake video filenames from metadata
    real_videos = [video for video, info in metadata.items() if info['label'] == 'REAL']
    fake_videos = [video for video, info in metadata.items() if info['label'] == 'FAKE']

    # Select the desired number of videos
    selected_real_videos = real_videos[:num_real]
    selected_fake_videos = fake_videos[:num_fake]

    # Combine selected videos
    all_selected_videos = selected_real_videos + selected_fake_videos

    # Copy selected videos to the target directory
    for video in all_selected_videos:
        src = os.path.join(source_dir, video)
        dst = os.path.join(target_dir, video)
        if os.path.exists(src):
            shutil.copy(src, dst)
            print(f"Copied {video} to {target_dir}")
        else:
            print(f"Video not found: {src}")

# Load metadata.json
with open('train_sample_videos/metadata.json', 'r') as f:
    metadata = json.load(f)

# Define directories
source_dir = 'train_sample_videos'
target_dir = 'selected_videos'

# Filter and copy videos
filter_and_copy_videos(metadata, source_dir, target_dir)



In [ ]:

import shutil
import os


# Define source and target directories
source_dir = '/content/selected_videos'  # Update this to your Colab directory
target_dir = '/content/drive/MyDrive/10videos'  # Update this to your Google Drive directory

# Create the target directory in Google Drive if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Move files from Colab environment to Google Drive
for filename in os.listdir(source_dir):
    src_path = os.path.join(source_dir, filename)
    dst_path = os.path.join(target_dir, filename)
    shutil.move(src_path, dst_path)

print("Files moved to Google Drive successfully.")


In [ ]:
from google.colab import drive
import json
import os
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define directories
base_dir = '/content/drive/MyDrive/train_sample_videos'
source_dir = '/content/selected_videos'
selected_real_videos_dir = os.path.join(base_dir, 'selected_real_videos')
selected_fake_videos_dir = os.path.join(base_dir, 'selected_fake_videos')

# Load metadata.json
with open('/content/train_sample_videos/metadata.json', 'r') as f:
    metadata = json.load(f)

def filter_and_copy_videos(metadata, source_dir, num_real=5, num_fake=5):
    # Create target directory if it does not exist
    if not os.path.exists(source_dir):
        os.makedirs(source_dir)

    # Extract real and fake video filenames from metadata
    real_videos = [video for video, info in metadata.items() if info['label'] == 'REAL']
    fake_videos = [video for video, info in metadata.items() if info['label'] == 'FAKE']

    # Select the desired number of videos
    selected_real_videos = real_videos[:num_real]
    selected_fake_videos = fake_videos[:num_fake]

    # Combine selected videos
    all_selected_videos = selected_real_videos + selected_fake_videos

    # Copy selected videos to the target directory
    for video in all_selected_videos:
        src = os.path.join('train_sample_videos', video)
        dst = os.path.join(source_dir, video)
        if os.path.exists(src):
            shutil.copy(src, dst)
            print(f"Copied {video} to {source_dir}")
        else:
            print(f"Video not found: {src}")

# Filter and copy videos
filter_and_copy_videos(metadata, source_dir)

# Create directories in Google Drive for real and fake videos
os.makedirs(selected_real_videos_dir, exist_ok=True)
os.makedirs(selected_fake_videos_dir, exist_ok=True)

# Separate the copied videos into real and fake directories
for filename in os.listdir(source_dir):
    src_path = os.path.join(source_dir, filename)
    if filename in metadata:
        label = metadata[filename]['label']
        if label == 'REAL':
            dst_path = os.path.join(selected_real_videos_dir, filename)
        elif label == 'FAKE':
            dst_path = os.path.join(selected_fake_videos_dir, filename)
        else:
            print(f"Unknown label for {filename}: {label}")
            continue
        shutil.move(src_path, dst_path)
        print(f"Moved {filename} to {dst_path}")

print("Videos separated and moved to Google Drive successfully.")


In [ ]:
import os
import cv2

# Define directories for frames
frames_real_videos_dir = os.path.join(base_dir, 'frames_real_videos')
frames_fake_videos_dir = os.path.join(base_dir, 'frames_fake_videos')

# Create directories for frames in Google Drive
os.makedirs(frames_real_videos_dir, exist_ok=True)
os.makedirs(frames_fake_videos_dir, exist_ok=True)

def extract_frames(video_path, frames_output_dir):
    """
    Extracts frames from a video and saves them to the specified directory.
    """
    try:
        vidcap = cv2.VideoCapture(video_path)
        if not vidcap.isOpened():
            print(f"Error: Unable to open video file {video_path}")
            return
        count = 0
        success, image = vidcap.read()
        while success:
            frame_filename = os.path.join(frames_output_dir, f"frame_{count:04d}.jpg")
            cv2.imwrite(frame_filename, image)  # Save frame as JPEG file
            success, image = vidcap.read()
            count += 1
        vidcap.release()
        print(f"Frames extracted and saved to {frames_output_dir}")
    except Exception as e:
        print(f"Error extracting frames from {video_path}: {e}")

# Extract frames from real videos
for filename in os.listdir(selected_real_videos_dir):
    video_path = os.path.join(selected_real_videos_dir, filename)
    frames_output_dir = os.path.join(frames_real_videos_dir, os.path.splitext(filename)[0])
    os.makedirs(frames_output_dir, exist_ok=True)  # Create directory for each video's frames
    extract_frames(video_path, frames_output_dir)

# Extract frames from fake videos
for filename in os.listdir(selected_fake_videos_dir):
    video_path = os.path.join(selected_fake_videos_dir, filename)
    frames_output_dir = os.path.join(frames_fake_videos_dir, os.path.splitext(filename)[0])
    os.makedirs(frames_output_dir, exist_ok=True)  # Create directory for each video's frames
    extract_frames(video_path, frames_output_dir)

print("Frame extraction completed and saved to Google Drive.")


In [ ]:
import os
import cv2
import numpy as np

# Define directories for frames
frames_real_videos_dir = '/content/drive/MyDrive/train_sample_videos/frames_real_videos'
frames_fake_videos_dir = '/content/drive/MyDrive/train_sample_videos/frames_fake_videos'

# Define a directory to save preprocessed frames
preprocessed_real_videos_dir = '/content/drive/MyDrive/train_sample_videos/preprocessed_real_videos'
preprocessed_fake_videos_dir = '/content/drive/MyDrive/train_sample_videos/preprocessed_fake_videos'

# Create directories for preprocessed frames
os.makedirs(preprocessed_real_videos_dir, exist_ok=True)
os.makedirs(preprocessed_fake_videos_dir, exist_ok=True)

def preprocess_frame(frame):
    """
    Preprocesses a single frame. This example includes resizing and normalization.
    Modify this function according to your preprocessing needs.
    """
    # Resize frame to 224x224 (example size, adjust as needed)
    frame = cv2.resize(frame, (224, 224))

    # Normalize frame to [0, 1]
    frame = frame / 255.0

    return frame

def process_frames(frames_dir, preprocessed_dir):
    """
    Processes frames from the given directory and saves preprocessed frames.
    """
    for video_folder in os.listdir(frames_dir):
        video_frames_dir = os.path.join(frames_dir, video_folder)
        preprocessed_video_frames_dir = os.path.join(preprocessed_dir, video_folder)
        os.makedirs(preprocessed_video_frames_dir, exist_ok=True)

        for frame_file in os.listdir(video_frames_dir):
            frame_path = os.path.join(video_frames_dir, frame_file)
            # Read the frame
            frame = cv2.imread(frame_path)
            if frame is not None:
                # Preprocess the frame
                preprocessed_frame = preprocess_frame(frame)
                # Save the preprocessed frame
                preprocessed_frame_path = os.path.join(preprocessed_video_frames_dir, frame_file)
                cv2.imwrite(preprocessed_frame_path, (preprocessed_frame * 255).astype(np.uint8))  # Convert back to [0, 255]

# Process and preprocess frames for real and fake videos
process_frames(frames_real_videos_dir, preprocessed_real_videos_dir)
process_frames(frames_fake_videos_dir, preprocessed_fake_videos_dir)

print("Frame preprocessing completed and saved to Google Drive.")


In [ ]:
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define directories in Google Drive and Colab environment
preprocessed_real_videos_dir = '/content/drive/MyDrive/train_sample_videos/preprocessed_real_videos'
preprocessed_fake_videos_dir = '/content/drive/MyDrive/train_sample_videos/preprocessed_fake_videos'
colab_real_videos_dir = '/content/preprocessed_real_videos'
colab_fake_videos_dir = '/content/preprocessed_fake_videos'

# Create directories in Colab
os.makedirs(colab_real_videos_dir, exist_ok=True)
os.makedirs(colab_fake_videos_dir, exist_ok=True)

# Copy preprocessed frames from Google Drive to Colab
shutil.copytree(preprocessed_real_videos_dir, colab_real_videos_dir, dirs_exist_ok=True)
shutil.copytree(preprocessed_fake_videos_dir, colab_fake_videos_dir, dirs_exist_ok=True)

print("Preprocessed frames copied to Colab environment.")


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None):
        self.real_dir = real_dir

        self.fake_dir = fake_dir
        self.transform = transform

        self.real_images = self._load_images(self.real_dir)
        self.fake_images = self._load_images(self.fake_dir)

        self.all_images = self.real_images + self.fake_images
        self.labels = [1] * len(self.real_images) + [0] * len(self.fake_images)

    def _load_images(self, dir_path):
        images = []
        for subdir, _, files in os.walk(dir_path):
            for file in files:
                file_path = os.path.join(subdir, file)
                try:
                    img = Image.open(file_path).convert('RGB')
                    images.append(img)
                except Exception as e:
                    print(f"Error loading image {file_path}: {e}")
        return images

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        img = self.all_images[idx]
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to fit the GAN's input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Instantiate dataset and dataloader
dataset = CustomDataset(preprocessed_real_videos_dir, preprocessed_fake_videos_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os

# Generator Network
class Generator(nn.Module):
    def __init__(self, input_dim, label_dim):
        super(Generator, self).__init__()
        self.label_dim = label_dim
        self.model = nn.Sequential(
            nn.Linear(input_dim + label_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 64*64*3),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        input = torch.cat((noise, labels), dim=1)
        output = self.model(input)
        output = output.view(-1, 3, 64, 64)
        return output

# Discriminator Network
class Discriminator(nn.Module):
    def __init__(self, label_dim):
        super(Discriminator, self).__init__()
        self.label_dim = label_dim
        self.features = nn.Sequential(
            nn.Linear(64*64*3 + label_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        self.feature_layer = nn.Sequential(
            nn.Linear(64*64*3 + label_dim, 512),
            nn.LeakyReLU(0.2)
        )

    def forward(self, imgs, labels):
        imgs_flat = imgs.view(imgs.size(0), -1)
        input = torch.cat((imgs_flat, labels), dim=1)
        features = self.feature_layer(input)
        output = self.features(input)
        return output, features


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data
import DataLoader

def train_cgan(generator, discriminator, dataloader, epochs=100, lr=0.0002, beta1=0.5, beta2=0.999, feature_csv='features.csv'):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  generator.to(device)
  discriminator.to(device)

# Define loss function and optimizers
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, beta2))
optimizer_d = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, beta2))

feature_list = []

for epoch in range(epochs):
    d_loss_real_total = 0
    d_loss_fake_total = 0
    g_loss_total = 0
    correct_preds = 0
    total_preds = 0

    for imgs, labels in dataloader:
        batch_size = imgs.size(0)
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        labels = labels.float().unsqueeze(1).to(device)

        # Train Discriminator
        optimizer_d.zero_grad()

        outputs, _ = discriminator(imgs.to(device), labels)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        noise = torch.randn(batch_size, 100, device=device)
        fake_imgs = generator(noise, labels)
        outputs, _ = discriminator(fake_imgs.detach(), fake_labels)
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()

        d_loss = d_loss_real + d_loss_fake
        optimizer_d.step()

        d_loss_real_total += d_loss_real.item()
        d_loss_fake_total += d_loss_fake.item()

        # Track discriminator accuracy
        preds = (outputs > 0.5).float()
        correct_preds += (preds == real_labels).sum().item()
        total_preds += real_labels.size(0)

        # Train Generator
        optimizer_g.zero_grad()

        outputs, _ = discriminator(fake_imgs, real_labels)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()

        optimizer_g.step()

        g_loss_total += g_loss.item()

    # Calculate average losses and accuracy
    avg_d_loss_real = d_loss_real_total / len(dataloader)
    avg_d_loss_fake = d_loss_fake_total / len(dataloader)
    avg_d_loss = (avg_d_loss_real + avg_d_loss_fake) / 2
    avg_g_loss = g_loss_total / len(dataloader)
    accuracy = (correct_preds / total_preds) * 100

    print(f'Epoch [{epoch+1}/{epochs}] | D Loss: {avg_d_loss:.4f} | G Loss: {avg_g_loss:.4f} | Accuracy: {accuracy:.2f}%')

# Save features to CSV
## This is formatted as code
all_features = np.vstack([item[0] for item in feature_list])
all_labels = np.hstack([item[1] for item in feature_list])
df = pd.DataFrame(all_features)
df['label'] = all_labels
df.to_csv(feature_csv, index=False)

In [ ]:
import pandas as pd
import h5py

def csv_to_h5(csv_file, h5_file='features.h5'):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Open an HDF5 file
    with h5py.File(h5_file, 'w') as hf:
        # Convert DataFrame to numpy array and save as a dataset
        hf.create_dataset('data', data=df.to_numpy())
        # Save column names as an attribute
        hf.attrs['columns'] = df.columns.tolist()

# Example usage
csv_file = '/content/features.csv'
csv_to_h5(csv_file)


In [ ]:
import h5py

def read_h5(h5_file='features.h5'):
    try:
        # Open an HDF5 file for reading
        with h5py.File(h5_file, 'r') as hf:
            # Access the dataset
            if 'data' in hf:
                data = hf['data'][:]
            else:
                raise KeyError("'data' dataset not found in the file.")

            # Access the attributes
            if 'columns' in hf.attrs:
                columns = hf.attrs['columns']
            else:
                raise KeyError("'columns' attribute not found in the file.")

            return data, columns

    except KeyError as e:
        print(f"Error: {e}")
        print(f"The specified object does not exist in the file: {h5_file}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
data, columns = read_h5()
print("Data:")
print(data)
print("Columns:")
print(columns)


In [ ]:
import torch
import numpy as np
import os
import h5py
from PIL import Image
from torchvision import transforms, models
import torch.nn as nn

# Define preprocessing transforms
preprocess = transforms.Compose([
    transforms.Resize((112, 112)),  # Adjust to match the model's input size
    transforms.ToTensor(),
])

def load_frames_from_directory(directory_path, num_frames=16):
    all_frames = []
    video_files = sorted(os.listdir(directory_path))  # Ensure consistent ordering
    for video_file in video_files:
        video_path = os.path.join(directory_path, video_file)
        frames = []
        for img_file in sorted(os.listdir(video_path)):  # Assuming frames are in subdirectories
            img_path = os.path.join(video_path, img_file)
            frame = Image.open(img_path).convert('RGB')  # Ensure 3 channels (RGB)
            frame = preprocess(frame)  # Apply preprocessing
            frames.append(frame)
            if len(frames) >= num_frames:
                break

        if len(frames) == num_frames:
            all_frames.append(torch.stack(frames))  # Shape: [num_frames, C, H, W]
        else:
            print(f"Not enough frames found in directory: {video_path}")

    return all_frames

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x: [batch_size, seq_length, input_dim]
        lstm_out, (hn, cn) = self.lstm(x)
        # Use the output from the last time step
        last_time_step_out = lstm_out[:, -1, :]  # Take the output from the last time step
        output = self.fc(last_time_step_out)
        return output

def preprocess_frames(frames, feature_extractor):
    features = []
    for frame in frames:
        frame = frame.unsqueeze(0)  # Add batch dimension
        with torch.no_grad():
            feature = feature_extractor(frame).squeeze()  # Extract features
        features.append(feature)
    return torch.stack(features)  # Shape: [num_frames, num_features]

def extract_temporal_features(real_dir, fake_dir, feature_extractor, model):
    all_features = []
    all_labels = []

    # Extract features from real videos
    real_frames = load_frames_from_directory(real_dir)
    for frames in real_frames:
        features = preprocess_frames(frames, feature_extractor)
        features_tensor = features.unsqueeze(0).to(device)  # Shape: [1, num_frames, num_features]
        with torch.no_grad():
            output = model(features_tensor).cpu().numpy()
            all_features.append(output.flatten())
            all_labels.append(1)  # Label for real videos

    # Extract features from fake videos
    fake_frames = load_frames_from_directory(fake_dir)
    for frames in fake_frames:
        features = preprocess_frames(frames, feature_extractor)
        features_tensor = features.unsqueeze(0).to(device)  # Shape: [1, num_frames, num_features]
        with torch.no_grad():
            output = model(features_tensor).cpu().numpy()
            all_features.append(output.flatten())
            all_labels.append(0)  # Label for fake videos

    if len(all_features) == 0:
        raise ValueError("No features extracted. Check the input directories and video frames.")

    return np.array(all_features), np.array(all_labels)

def save_features_to_hdf5(features, labels, hdf5_file):
    with h5py.File(hdf5_file, 'w') as hf:
        hf.create_dataset('features', data=features)
        hf.create_dataset('labels', data=labels)

# Define directories
real_video_dir = '/content/preprocessed_real_videos'
fake_video_dir = '/content/preprocessed_fake_videos'

# Define dimensions
input_dim = 2048  # Example: Flattened features from a CNN
hidden_dim = 512
num_layers = 2
output_dim = 1

# Instantiate feature extractor (e.g., a pre-trained CNN like ResNet)
feature_extractor = models.resnet50(pretrained=True)
feature_extractor.fc = nn.Identity()  # Remove the final classification layer

# Instantiate the LSTM model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
feature_extractor = feature_extractor.to(device)
lstm_model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim).to(device)

# Extract features and save them
features, labels = extract_temporal_features(real_video_dir, fake_video_dir, feature_extractor, lstm_model)
save_features_to_hdf5(features, labels, 'temporal_features.h5')
print("Features and labels saved to 'temporal_features.h5'")


In [ ]:
import h5py

# Check keys in the HDF5 file
def check_hdf5_keys(file_path):
    with h5py.File(file_path, 'r') as file:
        print("Datasets in the file:", list(file.keys()))

# Paths to the HDF5 files
hdf5_file_1 = '/content/features.h5'
hdf5_file_2 = '/content/temporal_features.h5'

print("Checking first HDF5 file:")
check_hdf5_keys(hdf5_file_1)

print("Checking second HDF5 file:")
check_hdf5_keys(hdf5_file_2)


In [ ]:
import h5py
import numpy as np

# Paths to the two HDF5 files you want to fuse
hdf5_file_1 = '/content/features.h5'
hdf5_file_2 = '/content/temporal_features.h5'
combined_hdf5_file = '/content/drive/My Drive/new_combined_file.h5'  # Path in Google Drive

def load_dataset(file_path, dataset_name):
    try:
        with h5py.File(file_path, 'r') as file:
            if dataset_name in file:
                return file[dataset_name][:]
            else:
                raise KeyError(f"Dataset '{dataset_name}' does not exist in file '{file_path}'")
    except Exception as e:
        print(f"Error loading dataset '{dataset_name}' from file '{file_path}': {e}")
        return None

# List datasets in a file to debug
def list_datasets(file_path):
    with h5py.File(file_path, 'r') as file:
        print(f"Datasets in file {file_path}:")
        def printname(name):
            print(name)
        file.visit(printname)

# Load datasets from the first HDF5 file
list_datasets(hdf5_file_1)  # This will print all available datasets in features.h5

# Load datasets from the second HDF5 file
features_2 = load_dataset(hdf5_file_2, 'features')
labels_2 = load_dataset(hdf5_file_2, 'labels')

# Explicitly check if any dataset is None
if features_2 is None:
    raise ValueError("Dataset 'features' could not be loaded from the second file.")
if labels_2 is None:
    raise ValueError("Dataset 'labels' could not be loaded from the second file.")

# Load 'data' from the first HDF5 file
data_1 = load_dataset(hdf5_file_1, 'data')

# Check if 'data' from the first file is loaded
if data_1 is None:
    raise ValueError("Dataset 'data' could not be loaded from the first file.")

# Check shapes of the datasets
print("Features 2 shape:", features_2.shape)
print("Labels 2 shape:", labels_2.shape)
print("Data 1 shape:", data_1.shape)

# Ensure feature dimensions are aligned
feature_dim = min(features_2.shape[1], data_1.shape[1])

# Truncate the features to the minimum dimension size
features_2 = features_2[:, :feature_dim]
data_1 = data_1[:, :feature_dim]

# Combine the features
try:
    combined_features = np.concatenate((data_1, features_2), axis=0)
except ValueError as e:
    print(f"Error combining features: {e}")
    raise

# Combine the labels (ensure they are aligned with the combined features)
combined_labels = np.concatenate((labels_2, labels_2))  # Adjust this based on the actual labels' alignment

# Check the combined data
print("Combined features shape:", combined_features.shape)
print("Combined labels shape:", combined_labels.shape)

# Create a new HDF5 file and save the combined datasets
try:
    with h5py.File(combined_hdf5_file, 'w') as combined_file:
        # Save combined features
        combined_file.create_dataset('combined_features', data=combined_features)

        # Save combined labels
        combined_file.create_dataset('combined_labels', data=combined_labels)

    print("Data successfully combined and saved to:", combined_hdf5_file)

except Exception as e:
    print(f"Error saving combined data: {e}")


In [ ]:
import h5py

file_path = '/content/drive/MyDrive/new_combined_file.h5'

def list_datasets(file_path):
    with h5py.File(file_path, 'r') as file:
        print(f"Datasets in file {file_path}:")
        def printname(name):
            print(name)
        file.visit(printname)

list_datasets(file_path)  # This will print all available datasets in the HDF5 file


In [ ]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

# Define file path
file_path = '/content/drive/MyDrive/new_combined_file.h5'

# Load data
try:
    with h5py.File(file_path, 'r') as file:
        # List datasets to debug
        print("Datasets in file:")
        for name in file:
            print(name)

        # Load datasets with correct names
        features = file['combined_features'][:]  # Updated dataset name
        labels = file['combined_labels'][:]  # Updated dataset name

    # Print shapes to confirm
    print("Features shape:", features.shape)
    print("Labels shape:", labels.shape)

    # Ensure labels match the number of features
    min_length = min(features.shape[0], len(labels))
    features = features[:min_length]
    labels = labels[:min_length]

    # Convert labels to binary if necessary
    # Example: Assuming labels should be 0 or 1
    labels = (labels > 0.5).astype(int)  # Adjust this based on your label encoding

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Print shapes for training and testing data
    print(f"Training features shape: {X_train.shape}")
    print(f"Training labels shape: {y_train.shape}")
    print(f"Test features shape: {X_test.shape}")
    print(f"Test labels shape: {y_test.shape}")

except KeyError as e:
    print(f"KeyError: {e} - Check if the dataset names are correct.")
except Exception as e:
    print(f"Error: {e}")


In [ ]:
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
file_path = '/content/drive/MyDrive/new_combined_file.h5'

with h5py.File(file_path, 'r') as file:
    features = file['/combined_features'][:]
    labels = file['/combined_labels'][:]

# Inspect shapes
print("Features shape:", features.shape)
print("Labels shape:", labels.shape)

# Ensure number of labels matches number of features
num_features = features.shape[0]
num_labels = labels.shape[0]

if num_features != num_labels:
    min_length = min(num_features, num_labels)
    features = features[:min_length]
    labels = labels[:min_length]

# Convert labels to binary if necessary
# Example: Assuming labels should be 0 or 1
labels = (labels > 0.5).astype(int)  # Adjust this based on your label encoding

# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Define a more complex neural network
class ImprovedNN(nn.Module):
    def __init__(self):
        super(ImprovedNN, self).__init__()
        self.fc1 = nn.Linear(1, 512)  # Increased number of units
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)  # Dropout for regularization
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

# Create datasets and loaders
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                              torch.tensor(y_train, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                             torch.tensor(y_test, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize model, loss function, and optimizer
model = ImprovedNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(20):  # Increased number of epochs
    model.train()
    running_loss = 0.0
    for data, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(data).squeeze()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, targets in test_loader:
        outputs = model(data).squeeze()
        predicted = (outputs > 0.5).float()
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

    accuracy = correct / total
    print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
# Save the model
torch.save(model.state_dict(), 'simple_nn.pth')
print("Model saved to 'simple_nn.pth'")


In [ ]:
def save_model_to_h5(model, pth_path, h5_path):
    # Load the state dict
    state_dict = torch.load(pth_path)

    # Create an HDF5 file
    with h5py.File(h5_path, 'w') as f:
        for key, value in state_dict.items():
            # Convert tensor to numpy array and save to HDF5 dataset
            f.create_dataset(key, data=value.cpu().numpy())

# Save the model to HDF5 format
save_model_to_h5(model, 'simple_nn.pth', 'new_modell.h5')
print("Model saved to 'model.h5'")

In [ ]:
import h5py

def explore_hdf5(file_path):
    try:
        # Open the HDF5 file in read-only mode
        with h5py.File(file_path, 'r') as file:
            # Print all root level object names (aka keys)
            print("Root Keys:", list(file.keys()))

            # Iterate over top-level groups and datasets
            for key in file.keys():
                print(f"\nExploring '{key}'")
                item = file[key]
                if isinstance(item, h5py.Group):
                    print(f"Group '{key}' contains:")
                    for subkey in item.keys():
                        print(f"  Dataset '{subkey}' with shape {item[subkey].shape}")
                elif isinstance(item, h5py.Dataset):
                    print(f"Dataset '{key}' with shape {item.shape}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Replace with the actual path to your HDF5 file
file_path = '/content/drive/MyDrive/new_combined_file.h5'
explore_hdf5(file_path)
